In [2]:
import os
os.chdir("..")
print(os.getcwd())

q:\AD project


## TO-DOS for today
1. Build the vector database
2. Build the basic RAG pipeline
3. HyDE and Modified HyDE integrations

## BUILD VECTOR DATABASE

## GET THE DATA

In [3]:
# BOOK DATA
from src.book_preprocess import get_book_data

book_doc_data = get_book_data(100)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 23.10it/s]


In [4]:
len(book_doc_data)

3049

In [5]:
import json

with open('artifacts\YouTube_API_Transcripts\chunked_transcripts_mba.json', 'r') as file:
    mba_data = json.load(file)

with open('artifacts\YouTube_API_Transcripts\chunked_transcripts_undergrad.json', 'r') as file:
    undergrad_data = json.load(file)

with open('artifacts\YouTube_API_Transcripts\chunked_misc_transcripts.json', 'r') as file:
    misc_data = json.load(file)

In [6]:
mba_data.keys()

dict_keys(['2S_vt-N0czE', '5T2PmzsxhkM', 'lnPfpvUJEIc', '44Gv-gJxAeo', 'BA3SbC9cFIo', 't3M26iDiEfo', 'iYMsQeChsp4', 'QKQtAGmmP-4', 'eBDAnoxRKIg', 'h8AnJ3deIto', 'H5jLvqp4vas', 's4khp86YR_I', 'vXxN6A88SA4', 'CBM5iXS2pNE', 'flZcxlRtxhg', 'o7HirKysCKQ', 'F3RBKPkDQHY', 'muu0lpWns-0', 'O-sQYbULzwg', 'IFWkYXBzcpw', 'X5uFWJljqYk', 'XNQiZjoZcqY', 'gzJq6bbW-Kk', 'DQDCQxyxJ9U', 'veCrq5J4MyY', '9kid8pL8YDo'])

In [30]:
from llama_index.core.schema import Document

all_data_list = []

curr_book = ""
for book_doc in book_doc_data:
    # try:
    if book_doc=={}: continue
    book = book_doc['book_source']
    if book!= curr_book:
        # print(book_doc)

        curr_book = book
        all_data_list.append(
            Document(
                text=book_doc['text'],
                metadata={
                    'page_num_coordinates':str(book_doc['page_num_coordinates']),
                    'book_source':book_doc['book_source'],
                },
                excluded_embed_metadata_keys=['page_num_coordinates','book_source'],
                excluded_llm_metadata_keys=['page_num_coordinates','book_source'],
            )
        )

for json_data in [undergrad_data,mba_data,misc_data]:
    for youtube_id, text_list in json_data.items():
        # print(youtube_id)
        all_data_list.append(
            Document(
                text=text_list[0]['text'],
                metadata={
                    "youtube_id":youtube_id,
                    "start_timestamp":text_list[0]['start_time'],
                },
                excluded_embed_metadata_keys=['youtube_id','start_timestamp'],
                excluded_llm_metadata_keys=['youtube_id','start_timestamp'],
            )
        )
        break

In [31]:
from llama_index.core import Document

Document(
    text="text",
    metadata={"hello":1}
)

Document(id_='440bd3d3-3474-4e4e-876c-ffd3f83d5d17', embedding=None, metadata={'hello': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='text', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [32]:
len(all_data_list)

12

In [33]:
all_data_list

[Document(id_='3e74e5f2-8dc7-4f48-b502-0bfc067f36c9', embedding=None, metadata={'page_num_coordinates': "[[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]]", 'book_source': 'Applied_corporate_finance'}, excluded_embed_metadata_keys=['page_num_coordinates', 'book_source'], excluded_llm_metadata_keys=['page_num_coordinates', 'book_source'], relationships={}, text='Let me begin this preface with a confession of a few of my own biases.\nFirst, I believe that theory, and the models that flow from it, should provide us with the tools to understand, analyze and solve problems.\nThe test of a model or theory then should not be based upon its elegance but upon its usefulness in problem solving.\nSecond, there is little in corporate financial theory, in my view, that is new and revolutionary.\nThe core principles of corporate finance are common sense ones, and have changed little over

In [34]:
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

In [21]:
import chromadb
COLLECTION_NAME = "ad-project"
chroma_client = chromadb.PersistentClient(
    path=COLLECTION_NAME,
)

In [22]:
# !pip install llama-index-vector-stores-chroma

In [35]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import Markdown, display
import chromadb

COLLECTION_NAME = "ad-project"
EMBEDDING_MODEL = "text-embedding-3-small"

ad_project_db = chromadb.PersistentClient(path="ad_project_db")
ad_project_chroma_collection = ad_project_db.get_or_create_collection(COLLECTION_NAME)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)
vector_store = ChromaVectorStore(chroma_collection=ad_project_chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    all_data_list, storage_context=storage_context, embed_model=embed_model
)

In [24]:
all_data_list[0]

Document(id_='ec1dd6b4-8929-4943-a87d-f60a6a5cd709', embedding=None, metadata={'page_num_coordinates': [[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]], 'book_source': 'Applied_corporate_finance'}, excluded_embed_metadata_keys=['page_num_coordinates', 'book_source'], excluded_llm_metadata_keys=['page_num_coordinates', 'book_source'], relationships={}, text='Let me begin this preface with a confession of a few of my own biases.\nFirst, I believe that theory, and the models that flow from it, should provide us with the tools to understand, analyze and solve problems.\nThe test of a model or theory then should not be based upon its elegance but upon its usefulness in problem solving.\nSecond, there is little in corporate financial theory, in my view, that is new and revolutionary.\nThe core principles of corporate finance are common sense ones, and have changed little over ti

In [27]:
s = str([[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]])
s

"[[{'page_num': 2}, [248.09210205078125, 282.16666666666674], [248.09210205078125, 1008.8333333333334], [1459.0, 1008.8333333333334], [1459.0, 282.16666666666674]]]"

In [29]:
import ast

data = ast.literal_eval(s)
type(data)

list

In [36]:
query_engine = index.as_query_engine()
response = query_engine.query("First steps in valuation")

In [37]:
response

Response(response='The first steps in valuation involve understanding the key drivers of value for the asset being evaluated. By identifying these key drivers, one can begin to assess the worth of the asset and make informed investment decisions. Valuation may seem complex, but with time spent collecting and analyzing information, anyone can learn to value assets effectively. It is essential to grasp the value drivers specific to each company or asset, as they vary depending on the nature of the business or property. Successful valuation requires a clear understanding of what creates value in a particular asset and being able to assess whether the price being paid aligns with its true worth.', source_nodes=[NodeWithScore(node=TextNode(id_='94bbed93-9262-4095-a320-147b4406ad6d', embedding=None, metadata={'page_num_coordinates': "[[{'page_num': 17}, [237.5, 831.5659722222222], [237.5, 1082.3159722222222], [971.8928472222221, 1082.3159722222222], [971.8928472222221, 831.5659722222222]], [

In [39]:
retriever = index.as_retriever()
nodes = retriever.retrieve("First step in valuations")

In [41]:
for node in nodes:
    print(node.metadata)

{'page_num_coordinates': "[[{'page_num': 17}, [237.5, 831.5659722222222], [237.5, 1082.3159722222222], [971.8928472222221, 1082.3159722222222], [971.8928472222221, 831.5659722222222]], [{'page_num': 17}, [237.68055555555554, 1089.965972222222], [237.68055555555554, 1384.4302978515625], [971.9743055555557, 1384.4302978515625], [971.9743055555557, 1089.965972222222]], [{'page_num': 18}, [233.49282836914062, 275.73263888888874], [233.49282836914062, 440.3493055555556], [971.6874999999998, 440.3493055555556], [971.6874999999998, 275.73263888888874]], [{'page_num': 18}, [232.6602325439453, 447.9993055555556], [232.6602325439453, 827.9493055555561], [971.7158333333332, 827.9493055555561], [971.7158333333332, 447.9993055555556]], [{'page_num': 18}, [235.61825561523438, 835.5993055555562], [235.61825561523438, 1129.4159722222228], [971.7052083333333, 1129.4159722222228], [971.7052083333333, 835.5993055555562]], [{'page_num': 18}, [236.35011291503906, 1137.0659722222226], [236.35011291503906, 1

In [44]:
# ad_project_chroma_collection.query(
#     query_texts="First steps in valuation"
# )

In [1]:
import os
os.getcwd()

'/home/athekunal/AD-project/AD-Finance-Agent/rag'

In [11]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
import chromadb

In [12]:
from config import *

In [13]:
db2 = chromadb.PersistentClient(path="ad_project_db")
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)
print(COLLECTION_NAME)
chroma_collection = db2.get_or_create_collection(COLLECTION_NAME)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index_ = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

ad-project


In [14]:
retriever = index_.as_retriever(similarity_top_k=2)
nodes = retriever.retrieve("First step in valuations")

In [15]:
nodes

[NodeWithScore(node=TextNode(id_='94bbed93-9262-4095-a320-147b4406ad6d', embedding=None, metadata={'page_num_coordinates': "[[{'page_num': 17}, [237.5, 831.5659722222222], [237.5, 1082.3159722222222], [971.8928472222221, 1082.3159722222222], [971.8928472222221, 831.5659722222222]], [{'page_num': 17}, [237.68055555555554, 1089.965972222222], [237.68055555555554, 1384.4302978515625], [971.9743055555557, 1384.4302978515625], [971.9743055555557, 1089.965972222222]], [{'page_num': 18}, [233.49282836914062, 275.73263888888874], [233.49282836914062, 440.3493055555556], [971.6874999999998, 440.3493055555556], [971.6874999999998, 275.73263888888874]], [{'page_num': 18}, [232.6602325439453, 447.9993055555556], [232.6602325439453, 827.9493055555561], [971.7158333333332, 827.9493055555561], [971.7158333333332, 447.9993055555556]], [{'page_num': 18}, [235.61825561523438, 835.5993055555562], [235.61825561523438, 1129.4159722222228], [971.7052083333333, 1129.4159722222228], [971.7052083333333, 835.59

In [19]:
from rag.frozen_rag import main_frozen_rag_answer

main_frozen_rag_answer("What are the first steps in valuation?")

ImportError: attempted relative import with no known parent package

In [19]:
import ast

metadata = ["[[{'page_num': 17}, [237.5, 831.5659722222222], [237.5, 1082.3159722222222], [971.8928472222221, 1082.3159722222222], [971.8928472222221, 831.5659722222222]], [{'page_num': 17}, [237.68055555555554, 1089.965972222222], [237.68055555555554, 1384.4302978515625], [971.9743055555557, 1384.4302978515625], [971.9743055555557, 1089.965972222222]], [{'page_num': 18}, [233.49282836914062, 275.73263888888874], [233.49282836914062, 440.3493055555556], [971.6874999999998, 440.3493055555556], [971.6874999999998, 275.73263888888874]], [{'page_num': 18}, [232.6602325439453, 447.9993055555556], [232.6602325439453, 827.9493055555561], [971.7158333333332, 827.9493055555561], [971.7158333333332, 447.9993055555556]], [{'page_num': 18}, [235.61825561523438, 835.5993055555562], [235.61825561523438, 1129.4159722222228], [971.7052083333333, 1129.4159722222228], [971.7052083333333, 835.5993055555562]], [{'page_num': 18}, [236.35011291503906, 1137.0659722222226], [236.35011291503906, 1344.749305555556], [971.6627083333334, 1344.749305555556], [971.6627083333334, 1137.0659722222226]], [{'page_num': 19}, [257.3603820800781, 311.52679443359375], [257.3603820800781, 476.4604166666667], [941.3176879882812, 476.4604166666667], [941.3176879882812, 311.52679443359375]], [{'page_num': 25}, [228.84576416015625, 1055.1770833333333], [228.84576416015625, 1392.371826171875], [971.5794444444443, 1392.371826171875], [971.5794444444443, 1055.1770833333333]], [{'page_num': 26}, [236.5471954345703, 275.73263888888874], [236.5471954345703, 784.8826388888893], [971.7158333333336, 784.8826388888893], [971.7158333333336, 275.73263888888874]]]"]

for md in metadata:
    pass

In [18]:

for node in nodes:
    print(node.metadata)
    print('-'*100)

NameError: name 'nodes' is not defined

In [70]:
%load_ext autoreload
%autoreload 2
from rag.frozen_rag import get_context, get_openai_answer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
context,metadata = get_context(question="First steps in valuation")

In [72]:
context

"DO YOU KNOW WHAT A SHARE IN GOOGLE OR APPLE is really worth? What about that condo or house you just bought? Should you care? Knowing the value of a stock, bond, or property may not be a prerequisite for successful investing, but it does help you make more informed judgments.Most investors see valuing an asset as a daunting task—something far too complex and complicated for their skill sets.\nConsequently, they leave it to the professionals (equity research analysts, appraisers) or ignore it entirely.\nI believe that valuation, at its core, is simple, and anyone who is willing to spend time collecting and analyzing information can do it.\nI show you how in this book.I also hope to strip away the mystique from valuation practices and provide ways in which you can look at valua- tion judgments made by analysts and appraisers and decide for yourself whether they make sense or not.While valuation models can be filled with details, the value of any company rests on a few key drivers, which

In [68]:
question="First steps in valuation"
out = get_openai_answer(question,context)

In [69]:
out

'The first steps in valuation involve understanding the value drivers of a business and collecting and analyzing information. Valuation is not a complex task and can be done by anyone willing to spend time on it. Valuation models may have details, but the value of a company rests on a few key drivers. These drivers vary from company to company and can be found by looking across different sectors and life cycles of companies. By understanding the value drivers, one can also identify investment bargains. The goal is to assess the value of any company or business that one is interested in buying and use this understanding to become a more informed and successful investor. Valuation is important because an investor should not pay more for an asset than it is worth. The price of a stock cannot be justified by the argument that there will be other investors who will pay a higher price in the future. Every asset, financial as well as real, has a value, and the key to successful investing lies

In [1]:
!pip install tiktoken


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from database import get_book_transcripts_data

doc_list = get_book_transcripts_data()

100%|██████████| 10/10 [00:00<00:00, 12.75it/s]


In [6]:
text_list = [dl.text for dl in doc_list]

In [7]:
len(text_list)

3128

In [2]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

6

In [9]:
from tqdm import tqdm
total_num_tokens = 0

for tl in tqdm(text_list):
    curr_tokens = num_tokens_from_string(tl,"cl100k_base")
    total_num_tokens += curr_tokens

100%|██████████| 3128/3128 [00:00<00:00, 3279.45it/s]


In [13]:
total_num_tokens

1819059

In [11]:
(total_num_tokens/1000)*0.00013

0.23647766999999997

In [12]:
(total_num_tokens/1000)*0.00002 

0.03638118

In [2]:
# from database import create_database

# create_database()

## MODIFIED HYDE

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("arsrira/ad-distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\astar\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\astar\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="arsrira/ad-distilgpt2")

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [22]:
def generate_answer(question):
    out = pipe(question,max_new_tokens=100,do_sample=True,min_new_tokens=10)
    return out[0]['generated_text']

In [23]:
# out = pipe("First steps in valuation")
# print(out[0]['generated_text'])

print(generate_answer("First steps in valuation"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


First steps in valuation are as follows:Step 1: Estimate the present value of cash flows and discount rates to arrive at the cash flows.Step 2: If all cash flows are viewed the same in terms of stable growth, estimate the required return on investments to discount the required return on equity.Step 3: Estimate the expected growth rate for each cash flow at each step of the process and the appropriate risk premium in the model.Assign a risk premium to each cash flow.Step 4: Adjust for risk


In [15]:
generate_answer("What is the second step in valuation?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the second step in valuation? A firm that is truly on the verge of declining rapidly should have a much better chance of regressing earnings and cash flows at the company level.If the current financial statements reflect the time horizon of the decline,'

In [16]:
generate_answer("What is the second step in valuation")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'What is the second step in valuation that matters? One is to assess the value that will be generated by the investment.For example, the value of a Home Depot computer has to be estimated directly by looking at the average annual cash flows (EBIT'

In [17]:
generate_answer("second step in valuation")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'second step in valuation is to value the equity in each holding and compute a total beta.Value each of the shares with $10 that comes with $10 expected in unlevered beta.Each of the shares with $10 expected growth potential can therefore be'

## RAG PIPELINE

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
import chromadb
FROM CINFIG